In [7]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [8]:
result_dirs = [x for x in Path("../../results/gradual/find_optimal_params").glob('nodes_*_neigh_*_cset_*')]
result_dirs

[PosixPath('../../results/gradual/find_optimal_params/nodes_8_neigh_6_cset_6'),
 PosixPath('../../results/gradual/find_optimal_params/nodes_20_neigh_5_cset_0'),
 PosixPath('../../results/gradual/find_optimal_params/nodes_11_neigh_8_cset_0'),
 PosixPath('../../results/gradual/find_optimal_params/nodes_11_neigh_4_cset_9'),
 PosixPath('../../results/gradual/find_optimal_params/nodes_8_neigh_8_cset_0'),
 PosixPath('../../results/gradual/find_optimal_params/nodes_11_neigh_7_cset_0'),
 PosixPath('../../results/gradual/find_optimal_params/nodes_11_neigh_5_cset_1'),
 PosixPath('../../results/gradual/find_optimal_params/nodes_20_neigh_4_cset_1'),
 PosixPath('../../results/gradual/find_optimal_params/nodes_20_neigh_3_cset_3'),
 PosixPath('../../results/gradual/find_optimal_params/nodes_20_neigh_3_cset_4')]

In [9]:
all_data = []

for result_dir in result_dirs:
    _, nodes, _, neigh, _, cset = result_dir.stem.split('_')
    nodes = int(nodes)
    neigh = int(neigh)
    cset = int(cset)

    edges = nodes

    try:
        data = pd.read_csv(result_dir / 'cpdag_metrics.csv')
    except FileNotFoundError:
        print(f"File not found: {result_dir / 'cpdag_metrics.csv'}")
        continue

    sid_low_values = data['sid_low'].values
    sid_high_values = data['sid_high'].values

    sid_low_norm = sid_low_values / edges
    sid_high_norm = sid_high_values / edges

    sid_low_mean = np.mean(sid_low_norm)
    sid_high_mean = np.mean(sid_high_norm)

    sid_low_std = np.std(sid_low_norm, ddof=1)
    sid_high_std = np.std(sid_high_norm, ddof=1)

    all_data.append({
        'nodes': nodes,
        'neigh': neigh,
        'cset': cset,
        'sid_low_mean': sid_low_mean,
        'sid_high_mean': sid_high_mean,
        'sid_low_std': sid_low_std,
        'sid_high_std': sid_high_std
    })


File not found: ../../results/gradual/find_optimal_params/nodes_11_neigh_8_cset_0/cpdag_metrics.csv
File not found: ../../results/gradual/find_optimal_params/nodes_20_neigh_3_cset_4/cpdag_metrics.csv


In [10]:
all_data = pd.DataFrame(all_data)
all_data.sort_values(by=['nodes', 'neigh', 'cset'], inplace=True)

In [11]:
all_data

,nodes,neigh,cset,sid_low_mean,sid_high_mean,sid_low_std,sid_high_std
0,8,6,6,3.087500,3.525000,1.070517,1.146856
3,8,8,0,2.650000,4.237500,1.949715,1.268543
2,11,4,9,3.654545,3.972727,0.926896,0.905092
5,11,5,1,3.518182,4.127273,0.914178,1.042882
4,11,7,0,2.663636,4.300000,1.042750,1.048896
7,20,3,3,6.405000,6.405000,1.750944,1.750944
6,20,4,1,6.535000,6.620000,1.742771,1.842733
1,20,5,0,5.935000,7.505000,1.737663,1.634600


In [12]:
best_by_nodes = all_data.loc[all_data.groupby('nodes')['sid_high_mean'].idxmin()]
best_by_nodes

,nodes,neigh,cset,sid_low_mean,sid_high_mean,sid_low_std,sid_high_std
0,8,6,6,3.087500,3.525000,1.070517,1.146856
2,11,4,9,3.654545,3.972727,0.926896,0.905092
7,20,3,3,6.405000,6.405000,1.750944,1.750944


Chosen params:

- 8 nodes:  neigh=6, cset=6
- 11 nodes: neigh=4, cset=9
- 20 nodes: neigh=3, cset=3